# Inference - sentiment and topics from product reviews and news articles

**Link:** https://learn.deeplearning.ai/courses/chatgpt-prompt-eng/lesson/5/inferring

In [1]:
from transformers import AutoTokenizer
import transformers
import torch

**Pipeline Workflow: Input -> Tokenization -> Model Inference -> Post-Processing (task dependent) -> Output**

In [2]:
def get_completion(prompt, model = "/kaggle/input/llama-2/pytorch/7b-chat-hf/1"):
    tokenizer = AutoTokenizer.from_pretrained(model)
    pipeline = transformers.pipeline(
        "text-generation",
        model=model,
        torch_dtype=torch.float16,
        device_map="auto",
    )
    sequences = pipeline(
        prompt,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_new_tokens=200,
    )
    #     for seq in sequences:
    #         print(f"Result: {seq['generated_text']}")
    return sequences

 > ***Inferencing from product reviews***

In [42]:
text_0 = f"""
I recently bought Sony's wireless noise-canceling headphones, and they're amazing for flights! \
The noise-canceling feature really blocks out the engine roar, making it so much easier to relax \
or focus on a movie. The sound quality is fantastic, with crisp highs and deep bass. They're comfortable to wear for long stretches too.\
The only downside is the price tag, but for frequent travelers, they're definitely worth it.
"""
text_1 = f"""
Avoid the FitFlop sandals! They claim they're super comfortable, but after a week my feet were killing me.\
The soles are hard and the angeblically supportive bumps felt more like torture than a massage.\
They also look clunky and didn't go well with anything in my wardrobe. Disappointed!
"""
for i in range(2):
    text = eval(f"text_{i}")
    prompt = f""" 
    Identify the following from the review which is delimited with triple quotes.:
    - Item purchased by reviewer
    - Is the reviewer expressing anger? (Boolean)
    - Brand Name (If not provided, write none)
    - Sentiment (positive or negative or neutral)
    \"\"\" {text} \"\"\"
    Format your response as JSON object with keys as: "Item","Anger","Brand","Sentiment"
    """
    response = get_completion(prompt)
    for res in response:
        print(res['generated_text'][len(prompt):])

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]


    {
    "Item": "Sony's wireless noise-canceling headphones",
    "Anger": False,
    "Brand": "Sony",
    "Sentiment": "positive"
    }





Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]


    Item: FitFlop sandals
    Anger: true
    Brand: FitFlop
    Sentiment: negative




# Transforming - translation, spelling error correction, converting code from 1 format to another

**Link:** https://learn.deeplearning.ai/chatgpt-prompt-eng/lesson/6/transforming

In [46]:
user_messages = [
    "Magical stay! The Evermore whispers stories in the halls. Breakfast on the rooftop overlooking the clouds was divine. 5 stars!",
    "जादुई अनुभव! एवरमोर की दीवारें कहानियाँ सुनाती हैं। बादलों को निहारते हुए छत पर नाश्ता अद्भुत था। 5 स्टार!",
    "L'Hôtel Clocktower est une merveille de design steampunk. Les engrenages tournent, les sifflets soufflent, mais les chambres sont étonnamment calmes. J'ai adoré le spectacle d'automates tous les soirs ! 4 étoiles (un peu cher).",
    "রোमाঞ্চকর অভিযান! ড্রাগনস ব্রেথ নিষ্ক্রিয় ज्वालामুখীতে নির্মিত। ম breathtaking দৃশ্য, কিন্তু लगातार গর্জন আমাকে বেশিরভাগ রাত জাগিয়ে রেখেছে। 3 তারকা (রোমাঞ্চ খোঁজা মানুষের জন্য ভাল, হালকা ঘুমের জন্য নয়)।",
    "¡Acogedor y encantador! Gnomebrook ofreció una estancia deliciosa. Habitaciones decoradas con muebles de hongos y deliciosas comidas a base de hongos. ¡Perfecto para los amantes de la naturaleza! 5 estrellas."
]
for text in user_messages:
    prompt = f"""
    Identify the following from the review which is delimited with triple quotes.:
    - Which Language is the text in?
    - What is the Hotel's name?
    - Translate the text to english
    - Sentiment (positive or negative or neutral)
    \"\"\" {text} \"\"\"
    Format your response as JSON object with keys as: "Language of original review","Hotel Name","English Translation","Sentiment".
    write only the json objects and don't write any code.
    """
    response = get_completion(prompt)
    for res in response:
        print(res['generated_text'][len(prompt):])

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]




{
"Language of original review": "English",
"Hotel Name": "Evermore",
"English Translation": "Magical stay! The Evermore whispers stories in the halls. Breakfast on the rooftop overlooking the clouds was divine. 5 stars!",
"Sentiment": "Positive"
}

Note that the language of the original review is English, the hotel name is "Evermore", the translation is "Magical stay! The Evermore whispers stories in the halls. Breakfast on the rooftop overlooking the clouds was divine. 5 stars!" and the sentiment of the review is positive.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]



{
"Language of original review": "Hindi",
"Hotel Name": "Evermore Resorts",
"English Translation": "Excellent experience! The storytelling of the walls were interesting and the staff were very helpful with the umbrellas.",
"Sentiment": "Positive"
}

please let me know if you need any additional information.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]


    {
    "Language of original review": "French",
    "Hotel Name": "L'Hôtel Clocktower",
    "English Translation": "The Hotel Clocktower is a steampunk design marvel. The gears turn, the whistles blow, but the rooms are surprisingly quiet. I loved the evening automaton show! 4 stars (bit pricey).",
    "Sentiment": "Positive"
    }

I have written only the JSON objects, please write the code to implement the logic to extract these details from the review. 


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]


    JSON Response:
    {
    "Language of original review": "Bengali",
    "Hotel Name": "Romantic Beach Hotel",
    "English Translation": "Breathtaking view, but the constant noise disturbs me a lot. 3 stars (not recommended for humans, not recommended for those who want peace, 3 stars for the views)",
    "Sentiment": "Negative"
    }
    


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]


    

{
"Language of original review": "Spanish",
"Hotel Name": "Gnomebrook",
"English Translation": "Warm and enchanting! Gnomebrook offered a delightful stay. Rooms decorated with fungus furniture and delicious fungus-based food. Perfect for nature lovers! 5 stars.",
"Sentiment": "Positive"
}









**JSON to HTML code**

In [54]:
data_json = {
  "books": [
    {
      "title": "The Hitchhiker's Guide to the Galaxy",
      "author": "Douglas Adams"
    },
    {
      "title": "Pride and Prejudice",
      "author": "Jane Austen"
    },
    {
      "title": "To Kill a Mockingbird",
      "author": "Harper Lee"
    }
  ]
}
prompt2 = f"""
Translate the following python dictionary from JSON to an HTML with column headers and title: {data_json}
"""
response2 = get_completion(prompt2)
for res in response2:
    print(res['generated_text'][len(prompt2):])

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]


Output:

<html>
<head>
<title>Books</title>
</head>
<body>
<table>
<tr>
<th>Title</th>
<th>Author</th>
</tr>
<tr>
<td>The Hitchhiker's Guide to the Galaxy</td>
<td>Douglas Adams</td>
</tr>
<tr>
<td>Pride and Prejudice</td>
<td>Jane Austen</td>
</tr>
<tr>
<td>To Kill a Mockingbird</td>
<td>Harper Lee</td>
</tr>
</table>
</body>
</html>

Explanation:

The JSON dictionary is translated into an HTML table with column headers and a title. The column headers are defined using


In [62]:
from IPython.display import display, Markdown, Latex, HTML, JSON
output = res['generated_text'][len(prompt2):]
start_tag = output.find("<html>")
end_tag = output.find("</html>")

if start_tag != -1 and end_tag != -1:
    content = output[start_tag:end_tag + len("</html>")]
    display(HTML(content))
else:
    print("HTML tags not found")

Title,Author
The Hitchhiker's Guide to the Galaxy,Douglas Adams
Pride and Prejudice,Jane Austen
To Kill a Mockingbird,Harper Lee
